### In this notebook we look into the robustness of the eigenvectors of the Hessian over the spectrum of eigenvalues with respect to a reference eigenvector.

In [ ]:
from __future__ import division
import os
import numpy as np
import time
import matplotlib.pyplot as plt
import scipy as scp
import pylab as pyl
import warnings
warnings.filterwarnings('ignore')
np.random.seed(1234)
plt.rcParams.update({'font.size': 8})

%matplotlib inline 

%load_ext autoreload                                                              
                                                                                                                                  
%autoreload 

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
relative_path_to_new_folder = "../Images"
os.makedirs(relative_path_to_new_folder, exist_ok=True)
if not os.path.isdir('../Images/Robustness_images'):
    os.makedirs('../Images/Robustness_images')

In [ ]:
"""To compute distance matrix"""
def distmat(x,y):
    return np.sum( x**2,0 )[:,None] + np.sum( y**2,0 )[None,:] - 2*x.transpose().dot(y)

"""To Normalise a vector"""
normalize = lambda a: a/np.sum( a )

"""To Compute P"""
def GetP(u,K,v):
    return u[:,None]*K*v[None,:]

def plotp(x, col,plt, scale=200, edgecolors="k"):
  return plt.scatter( x[0,:], x[1,:], s=scale, edgecolors=edgecolors,  c=col, cmap='plasma', linewidths=2 )

In [ ]:
def generate_data(N):
    """
     N is a list of the size of the data on x and y
    """
    x = np.random.rand( 2,N[0] )-0.5
    theta = 2*np.pi*np.random.rand( 1,N[1] )
    r = 0.8+.2*np.random.rand( 1,N[1] )
    y = np.vstack( ( r*np.cos( theta ),r*np.sin( theta ) ) )
    return x,y

In [ ]:
def spectral_decomposition(mat):
    eig, v = np.linalg.eigh( mat )
    sorting_indices = np.argsort(eig)
    eig = eig[sorting_indices]
    v   = v[:, sorting_indices]
    
    print( "List of smallest eigenvalues: ", eig[:10])
    print( "List of largest  eigenvalues: ", eig[-10:])

    return eig,v



In [ ]:
import computational_OT

In [ ]:
N = [ 500, 600 ]
x,y = generate_data(N)

### Log-domain Sinkhorn 

In [ ]:
a = normalize(np.ones(N[0]))
b = normalize(np.ones(N[1]))
# Log domain Sinkhorn
print("Log domain Sinkhorn.... ")
results_logSinkhorn = []
times_logSinkhorn   = []
logsinkhornP        = []
# epsilons = [ 0.1 , 0.05 , 0.01, 0.005,  0.001]
# epsilons = [1.0]

epsilons    = [ 0.5, 0.1, 0.05, 0.01, 0.005 ]
niter_array = [ 100, 200, 350, 2000, 4000]

#Cost matrix
C = distmat(x,y)

for i in range(len(epsilons)):
  eps   = epsilons[i]
  niter = niter_array[i]

  print( "Sinkhorn for epsilon = "+str(eps)+":" )

  print( "Doing for (",N[0],N[1],")." )
  print( " |- Iterating" )

  start = time.time()
  logsinkhorn = computational_OT.Log_domainSinkhorn(a,b,C,eps)
  output = logsinkhorn.update( niter = niter )
  results_logSinkhorn.append( output )
  end = time.time()
  times_logSinkhorn.append(1e-3*(end-start) )
  logsinkhornP.append(GetP(output['potential_f']/eps, np.exp(-C/eps),output['potential_g']/eps))

#### Error plot

In [ ]:
plt.figure( figsize = (20,7) )

plt.subplot(2,1,1),
plt.title( "$||P1 -a||_1+||P1 -b||_1$" )
for i in range( len( results_logSinkhorn) ):
  error = np.asarray( results_logSinkhorn[i]['error'] )
  plt.plot( error, label = 'log-sinkhorn for $\epsilon=$'+ str(epsilons[i]) , linewidth = 2 )
plt.yscale( 'log' )
plt.legend()
plt.xlabel("Iterations")
plt.ylabel("Error in log-scale")
plt.show()


In [ ]:
Hessians_logsinkhorn = []
for i in range(len(epsilons)):
    u  = np.exp((results_logSinkhorn[i]['potential_f'])/epsilons[i])  
    v  = np.exp((results_logSinkhorn[i]['potential_g'])/epsilons[i])
    K  = np.exp(-C/epsilons[i])
    r1 = u[:,None]*np.dot(K,v)[:,None] 
    r2 = v[:,None]*np.dot(K.T,u)[:,None]                                
    P  = u[:,None]*np.exp(-C/epsilons[i])*v[None,:]
    A  = np.diag( np.array(r1.reshape(r1.shape[0],)) )
    B  = P       
    C_  = P.T
    D  = np.diag( np.array(r2.reshape(r2.shape[0],)) )
    result = np.vstack( ( np.hstack((A,B)), np.hstack((C_,D)) ) )    
    Hessians_logsinkhorn.append(result)
    

In [ ]:
eigs = []
eigvecs = []
for i in range(len(epsilons)):
    eps = epsilons[i]
    print("Spectral statistics of Hessian for epsilon="+str(eps))
    diag   = 1/np.sqrt(np.diag( Hessians_logsinkhorn[i] ).flatten())
    result = diag[:,None]*Hessians_logsinkhorn[i]*diag[None,:]
    ev = spectral_decomposition( result )
    eigs.append(ev[0])
    eigvecs.append(ev[1])
    print("")


## Robustness plot of  eigenvectors


#### L2-norm

##### Middle reference epsilon

In [ ]:
plt.rcParams.update({'font.size': 20})
num = list(range(len(epsilons)-1))

error = {}
reference_epsilonindex = len(epsilons)//2
for i in range(10):
  last_eigvec = eigvecs[reference_epsilonindex][:,(N[0]+N[1])-i-1]

  error[i] = []
  for eps in range(len(epsilons)):
    if eps!= reference_epsilonindex:
      curr_eigvec = eigvecs[eps][:,(N[0]+N[1])-i-1]
      correlation = np.dot(curr_eigvec, last_eigvec) 
      if correlation<=0:
        curr_eigvec = -curr_eigvec
        sum_error = np.linalg.norm(curr_eigvec-last_eigvec)
      else:
        sum_error = np.linalg.norm(curr_eigvec-last_eigvec)
      error[i].append(sum_error)
  
xticks = []
for i in range(len(epsilons)):
  if i != reference_epsilonindex:
    xticks.append(str(epsilons[i])+"-"+str(epsilons[reference_epsilonindex]))

plt.figure(figsize= (30,30))
plt.subplot(2,1,1),
plt.title( "Robustness of eigenvectors among the different spectrum of eigenvalues corresponding to different"+" $\epsilon$") 
for i in range(10):
  plt.plot( num, error[i],label = str(i+1)+'th largest eigenvalue',
             linewidth = 2, marker = 'o', markersize = 20 )

plt.legend()
plt.ylabel("Error: L2 norm")
plt.xlabel("$\epsilon$ difference")
plt.xticks(list(range(len(xticks))),xticks)
plt.savefig("../Images/Robustness_images/Robustnessplotlargesteigenvalue_L2norm_midref.pdf",format = 'pdf', bbox_inches="tight")
plt.show()


##### End reference epsilon

In [ ]:
plt.rcParams.update({'font.size': 20})
num = list(range(len(epsilons)-1))

error = {}
reference_epsilonindex = len(epsilons)-1
for i in range(10):
  # last_eigv = eigvecs[-1][:,(N[0]+N[1])-i-1]
  last_eigvec = eigvecs[reference_epsilonindex][:,(N[0]+N[1])-i-1]

  error[i] = []
  for eps in range(len(epsilons)):
    if eps!= reference_epsilonindex:
      curr_eigvec = eigvecs[eps][:,(N[0]+N[1])-i-1]
      correlation = np.dot(curr_eigvec, last_eigvec) 
      if correlation<0:
        curr_eigvec = -curr_eigvec
        sum_error = np.linalg.norm(curr_eigvec-last_eigvec)
      else:
        sum_error = np.linalg.norm(curr_eigvec-last_eigvec)
      error[i].append(sum_error)
  
xticks = []
for i in range(len(epsilons)):
  if i != reference_epsilonindex:
    xticks.append(str(epsilons[i])+"-"+str(epsilons[reference_epsilonindex]))

plt.figure(figsize= (30,30))
plt.subplot(2,1,1),
plt.title( "Robustness of eigenvectors among the different spectrum of eigenvalues corresponding to different"+" $\epsilon$") 
for i in range(10):
  plt.plot( num, error[i],label = str(i+1)+'th largest eigenvalue',
             linewidth = 2, marker = 'o', markersize = 20 )

plt.legend()
plt.ylabel("Error: L2 norm")
plt.xlabel("$\epsilon$ difference")
plt.xticks(list(range(len(xticks))),xticks)
plt.savefig("../Images/Robustness_images/Robustnessplotlargesteigenvalue_L2norm_endref.pdf",format = 'pdf', bbox_inches="tight")
plt.show()

In [ ]:
eigenmodes_min   = 6
eigenmodes_max   = 10
eigenmodes_count = eigenmodes_max - eigenmodes_min + 1 
plt.rcParams.update({'font.size': 15})

X=np.hstack((np.hstack((x[0,:],y[0,:]))[:,None],np.hstack((x[1,:],y[1,:]))[:,None]))
fig, ax = plt.subplots(figsize=(30,40),nrows=len(epsilons), ncols = eigenmodes_count,sharey=True)
scale = 20
courant_modes = []
for i in range(len(epsilons)):
    eigenvectors = eigvecs[i]
    eigenvalues  = eigs[i]
    courant_modes.append( [] )
    for j in range(eigenmodes_min, eigenmodes_max+1):
        eigenvalue = eigenvalues[N[0]+N[1]-j-1]
        function_as_vector = eigenvectors[:,N[0]+N[1]-j-1]
        function_as_vector = function_as_vector/np.max(function_as_vector)
        # Index
        ax_index = j - eigenmodes_min
        # Flip sign if anti-correlated
        if i>0:
            old_mode = courant_modes[i-1][ax_index]
            correlation = np.dot( old_mode, function_as_vector)/(np.linalg.norm(old_mode)*np.linalg.norm(function_as_vector))
            if correlation<=0:
                    function_as_vector = -function_as_vector
        # Record
        courant_modes[i].append( function_as_vector )
        im1 = ax[i][ax_index].scatter(X.T[0,:], X.T[1,:], s=scale, edgecolors=(0,0,0,0),  c=function_as_vector, cmap='plasma', linewidths=2)
        fig.colorbar(im1,ax=ax[i][ax_index])
        # Title
        ax[i][ax_index].set_title( f'''${j}''' + "^{st}$ largest eig. $\lambda_{" + str(j) + "}$= " + str(np.round(eigenvalue, 3)) + f''', $\epsilon$={str(epsilons[i])} ''')
    # end for
# end for v

#fig.colorbar(im1, ax=ax.ravel().tolist())
plt.xlim(np.min(X.T[0,:])-.1,np.max(X.T[0,:])+.1)
plt.ylim(np.min(X.T[1,:])-.1,np.max(X.T[1,:])+.1)
plt.subplots_adjust(wspace=0,hspace=0.15)
plt.tight_layout()
plt.savefig("../Images/Robustness_images/Heatmapofeigvecof7largestEigvalL2.pdf", format = 'pdf')
plt.show();

